In [1]:
import json
import random
import re
from collections import Counter
from pathlib import Path

import pandas as pd

In [2]:
# TODO
# - agreement/consistency of model

In [3]:
STEP_NUM = {
            'A1': 4,
            'B1': 5,
            'C1': 10,
            'D1': 5,
        }

In [13]:
class AnalyseLLMParticipant:
    def __init__(self, output_file):
        self.output_file = output_file
        self.details, self.results = self.load_data()
        self.questions = self.load_questions()
        self.df = self.split_by_prompt()
        self.parse_generation_for_selection()
        self.response_eq_example()
        self.response_length_gt_steps()
        self.response_eq_all_steps()
        self.meta_object_selection()

    def load_data(self):
        with open(self.output_file) as f:
            data = json.load(f)

        detail = data['config']
        results = data['results']

        return detail, results

    def load_questions(self):

        questions = Path(f'../resources/data/select.json')
        with open(questions) as f:
            data = json.load(f)

        return data

    def split_by_prompt(self):

        qids = [r['question_id'] for r in self.results]
        templates = [r['template_id'] for r in self.results]
        responses = [r['response'] for r in self.results]

        system_prompt = 'system\n\n'
        user_prompt = "user\n\n"
        generation_prompt = 'assistant\n\n'

        system = []
        user = []
        generation = []

        for response in responses:
            system.append(response[len(system_prompt):response.index(user_prompt)])
            user.append(response[response.index(user_prompt) + len(user_prompt):response.index(generation_prompt)])
            generation.append(response[response.index(generation_prompt) + len(generation_prompt):])

        # dictionary with {qid: [system, user, generation]}
        return pd.DataFrame({'qid': qids, 'template': templates, 'num_steps': [STEP_NUM[t] for t in templates],
                             'system': system, 'user': user, 'generation': generation})

    def get_template_from_qid(
            self,
            qid,
            return_steps_or_id: str = 'steps',
        ):

        template = self.questions[qid]['template_id']

        if return_steps_or_id == 'steps':
            return STEP_NUM[template]
        else:
            return template


    def parse_generation_for_selection(self):

        def matches_format(generation):
            # regex for matching lists e.g., [1, 2, 3] or [a, b, c]
            r = re.compile(r'\[.*?\]')
            matches = r.findall(generation)
            if matches:
                match = matches[-1]
                try:
                    test = [str(m) for m in match[1:-1].split(',')]
                except ValueError:
                    test = None
                return match

        def valid_generation(template, generation):
            if generation is not None:
                try:
                    valid = [int(m) for m in generation[1:-1].split(',')]
                except ValueError:
                    valid = None
            else:
                valid = None

            # compare to number in different column
            if valid is not None and len(valid) <= STEP_NUM[template]:
                return valid

            return None

        self.df['matches_format'] = self.df['generation'].apply(lambda x: matches_format(x))
        # compare to number in df['num_steps']
        self.df['valid_generation'] = self.df[['template', 'matches_format']].apply(lambda x: valid_generation(x['template'], x['matches_format']), axis=1)


    def response_eq_example(self):

        example = self.details['EXAMPLE']
        # count number of times example == df['matches_format']
        self.df['response_eq_example'] = self.df['matches_format'].apply(lambda x: True if x == example else False)

    def response_length_gt_steps(self):

        # count cases where the assistant generation has more steps than the example
        self.df['response_length_gt_steps'] = self.df.apply(lambda row: True if row['valid_generation'] is not None and len(row['valid_generation']) > row['num_steps'] else False, axis=1)

    def response_eq_all_steps(self):

        # count cases where assistant generation is just the full list of steps
        self.df['response_eq_all_steps'] = self.df.apply(lambda row: True if row['valid_generation'] == [i for i in range(1, row['num_steps'] + 1)] else False, axis=1)

    def meta_object_selection(self):

        example = self.questions[list(self.questions.keys())[0]]
        xp = example['explanation']
        meta_steps = [i['step'] for i in xp if i['label'] == 'meta']
        object_steps = [i['step'] for i in xp if i['label'] == 'object']

        def meta_selections(generation):
            if generation is not None:
                return [i for i in generation if i in meta_steps]
            return None

        def meta_selections_pc(generation):
            if generation is not None:
                return len([i for i in generation if i in meta_steps]) / len(meta_steps)
            return None

        def object_selections(generation):
            if generation is not None:
                return [i for i in generation if i in object_steps]
            return None

        def object_selections_pc(generation):
            if generation is not None:
                return len([i for i in generation if i in object_steps]) / len(object_steps)
            return None

        self.df['meta_selections'] = self.df['valid_generation'].apply(lambda x: meta_selections(x))
        self.df['meta_selections_pc'] = self.df['valid_generation'].apply(lambda x: meta_selections_pc(x))
        self.df['object_selections'] = self.df['valid_generation'].apply(lambda x: object_selections(x))
        self.df['object_selections_pc'] = self.df['valid_generation'].apply(lambda x: object_selections_pc(x))


In [25]:
outputs = Path('outputs/select').iterdir()
output = sorted(outputs)[-1]
with output.open() as f:
    data = json.load(f)

config = data['config']
for k, v in config.items():
    print(f'{k}: {v}')

T = AnalyseLLMParticipant(output)
T.df = T.df[T.df['template'] == 'A1']

MODEL: /nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct
TEMPERATURE: 0.3
BATCH_SIZE: 20
EXAMPLE: [1, 2, 3, 4]
FORMAT_VERSION: 2
SYSTEM_CONTENT: Your role is to select, from a list the steps, those that are most important for inclusion in a summary explanation of that process. Format your output as a list, for example [1, 2, 3, 4]. Output only this short summary paragraph and nothing else.
INPUTS: ../resources/data/select.json
DESCRIPTION: testing


In [26]:
T.df['matches_format'].value_counts()

[1, 2, 3, 4]                    56
[1, 2, 3]                       13
[insert answer]                  2
[4]                              1
[1, 4]                           1
[approximately 11.3 million]     1
[1, 2, 4]                        1
[1, 3, 4]                        1
[1, 2, 9.8 million]              1
Name: matches_format, dtype: int64

In [27]:
T.df['valid_generation'].value_counts()

[1, 2, 3, 4]    56
[1, 2, 3]       13
[4]              1
[1, 4]           1
[1, 2, 4]        1
[1, 3, 4]        1
Name: valid_generation, dtype: int64

In [28]:
T.df['response_eq_example'].value_counts()

True     56
False    44
Name: response_eq_example, dtype: int64

In [29]:
T.df['response_length_gt_steps'].value_counts()

False    100
Name: response_length_gt_steps, dtype: int64

In [30]:
T.df['response_eq_all_steps'].value_counts()

True     56
False    44
Name: response_eq_all_steps, dtype: int64

In [31]:
T.df['meta_selections'].value_counts()

[1, 3]    70
[1]        2
[]         1
Name: meta_selections, dtype: int64

In [32]:
T.df['meta_selections_pc'].value_counts().sort_index()

0.000000     1
0.333333     2
0.666667    70
Name: meta_selections_pc, dtype: int64

In [33]:
T.df['object_selections'].value_counts()

[2, 4]    57
[2]       13
[4]        3
Name: object_selections, dtype: int64

In [34]:
T.df['object_selections_pc'].value_counts().sort_index()

0.5    16
1.0    57
Name: object_selections_pc, dtype: int64

In [35]:
# templates = ['A1', 'B1', 'C1', 'D1']

# output = Path('outputs/2024-05-24T16:16:02.060434.json')
# output = Path('outputs/2024-05-24T16:16:02.061578.json')
# # output = Path('outputs/2024-05-24T16:16:02.062049.json')
# with output.open() as f:
#     data = json.load(f)


# eq_example = []
# length_gt_steps = []
# eq_steps = []
# for t in templates:
#     T = PromptParticipant(output, t)
#     # count occurences != none
#     eq_example.append(sum(T.df['response_eq_example']))
#     length_gt_steps.append(sum(T.df['response_length_gt_steps']))
#     eq_steps.append(sum(T.df['response_eq_all_steps']))

# # create dataframe from results
# comp_df = pd.DataFrame({
#     'template': templates,
#     'num_steps': STEP_NUM.values(),
#     'eq_example': eq_example,
#     'length_gt_steps': length_gt_steps,
#     'eq_steps': eq_steps
# })

# print(comp_df)